In [1]:
use_tensorboard = False

if use_tensorboard:
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter("runs/vae_baseline") 

In [ ]:
train_langevine = False

In [2]:
import argparse
import json
import logging
import random
from datetime import datetime
from importlib import import_module
from itertools import chain
from os.path import join, exists

import matplotlib.pyplot as plt
import torch
import torch.backends.cudnn as cudnn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.utils.data import DataLoader
from torch.autograd import Variable

from new.aae.pcutil import plot_3d_point_cloud
#from utils.util import find_latest_epoch, prepare_results_dir, cuda_setup, setup_logging

cudnn.benchmark = True


In [3]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        gain = torch.nn.init.calculate_gain('relu')
        torch.nn.init.xavier_uniform_(m.weight, gain)
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.constant_(m.weight, 1)
        torch.nn.init.constant_(m.bias, 0)
    elif classname.find('Linear') != -1:
        gain = torch.nn.init.calculate_gain('relu')
        torch.nn.init.xavier_uniform_(m.weight, gain)
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)


In [4]:
random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed_all(123)

In [5]:
results_dir = "results/"

In [6]:
device = torch.device("cuda")

In [7]:
dataset_name = "shapenet"

In [8]:
from new.shapenet import ShapeNetDataset

In [9]:
dataset = ShapeNetDataset(root_dir="shapenet",
                          classes=["chair"])
points_dataloader = DataLoader(dataset, batch_size=16,
                               shuffle=True,
                               num_workers=8,
                               drop_last=True, pin_memory=True)

In [10]:
if train_langevine:
    from new.models import NetWrapper
    
    net = NetWrapper().to(device)
    net.apply(weights_init)

else:
    from new.aae.aae import Generator, Encoder

    G =  Generator().to(device)
    E = Encoder().to(device)

    G.apply(weights_init)
    E.apply(weights_init)

In [11]:
from new.params import *
from new.utils import *

In [12]:
#
# Float Tensors
#
fixed_noise = torch.FloatTensor(16, z_dim)
fixed_noise.normal_(mean=0, std=0.2)
std_assumed = torch.tensor(0.2)

fixed_noise = fixed_noise.to(device)
std_assumed = std_assumed.to(device)

In [13]:
#
# Optimizers
#
optim_params = {
                "lr": 0.0005,
                "weight_decay": 0,
                "betas": [0.9, 0.999],
                "amsgrad": False
            }

if train_langevine:
    optE = torch.optim.Adam(net.netE.parameters(), **optim_params)
    optG = torch.optim.Adam(net.netG.parameters(), **optim_params)
else:
    EG_optim = torch.optim.Adam(chain(E.parameters(), G.parameters()),
                        **optim_params)

In [14]:
from new.champfer_loss import ChamferLoss

reconstruction_loss = ChamferLoss().to(device)

In [ ]:
total_step = 0

In [ ]:
for epoch in range(100):
    start_epoch_time = datetime.now()
    
    if train_langevine:
        net.train()
    else:     
        G.train()
        E.train()

    total_loss = 0.0
    for i, point_data in enumerate(points_dataloader, 1):
#         if i > 1:
#             break
        total_step += 1
        
        X, _ = point_data
        X = X.to(device)

        # Change dim [BATCH, N_POINTS, N_DIM] -> [BATCH, N_DIM, N_POINTS]
        if X.size(-1) == 3:
            X.transpose_(X.dim() - 2, X.dim() - 1)

        if train_langevine:
            batch_num = X.shape[0]

            # Initialize chains
            z_g_0 = sample_p_0(n = batch_num,sig=e_init_sig, device = X.device)
            z_e_0 = sample_p_0(n = batch_num,sig=g_init_sig, device = X.device)

            # Langevin posterior and prior
            z_g_k = net(Variable(z_g_0), X, prior=False)
            z_e_k = net(Variable(z_e_0), prior=True)

            # reconstruction
            X_hat = net.netG(z_g_k.detach())
            loss_g = net.loss_fun(X_hat.transpose(1,2).contiguous() + 0.5, X.transpose(1,2).contiguous() + 0.5)

            # energy prior
            en_neg = net.netE(z_e_k.detach()).mean() # TODO(nijkamp): why mean() here and in Langevin sum() over energy? constant is absorbed into Adam adaptive lr
            en_pos = net.netE(z_g_k.detach()).mean()
            loss_e = en_pos - en_neg

            # Learn generator
            optG.zero_grad()
            loss_g.backward()
            optG.step()

            optE.zero_grad()
            loss_e.backward()
            # grad_norm_e = get_grad_norm(net.netE.parameters())
            # if args.e_is_grad_clamp:
            #    torch.nn.utils.clip_grad_norm_(net.netE.parameters(), args.e_max_norm)
            optE.step()


            if i % 30 == 0:
                print(f'[{epoch}: ({i})] '
                          f'loss_g: {loss_g.item():.4f} '
                          f'(loss_e: {loss_e.item(): .4f}'
                          f' Time: {datetime.now() - start_epoch_time}')


        else:
            codes, mu, logvar = E(X)
            X_rec = G(codes)

            loss_e = torch.mean(
                 0.05 *
                reconstruction_loss(X.permute(0, 2, 1) + 0.5,
                                    X_rec.permute(0, 2, 1) + 0.5))

            loss_kld = -0.5 * torch.mean(
                1 - 2.0 * torch.log(std_assumed) + logvar -
                (mu.pow(2) + logvar.exp()) / torch.pow(std_assumed, 2))

            loss_eg = loss_e + loss_kld
            EG_optim.zero_grad()
            E.zero_grad()
            G.zero_grad()

            loss_eg.backward()
            total_loss += loss_eg.item()
            EG_optim.step()

            if i % 30 == 0:
                if use_tensorboard:
                        writer.add_scalar('loss/Loss_EG',loss_eg.item(), total_step)
                        writer.add_scalar('loss/REC',loss_e.item(), total_step)
                        writer.add_scalar('loss/KLD',loss_kld.item(), total_step)
                else:
                    print(f'[{epoch}: ({i})] '
                              f'Loss_EG: {loss_eg.item():.4f} '
                              f'(REC: {loss_e.item(): .4f}'
                              f' KLD: {loss_kld.item(): .4f})'
                              f' Time: {datetime.now() - start_epoch_time}')

    print(
        f'[{epoch}/{400}] '
        f'Loss_G: {total_loss / i:.4f} '
        f'Time: {datetime.now() - start_epoch_time}'
    )
        
    ################################### eval ######################################
    #
    # Save intermediate results
    #
    if train_langevine:
        net.eval()
        with torch.no_grad():
            fake = net.netG(fixed_noise).data.cpu().numpy()
            X_rec = X_hat.data.cpu().numpy()
            X = X.data.cpu().numpy()
    else:
        G.eval()
        E.eval()
        with torch.no_grad():
            fake = G(fixed_noise).data.cpu().numpy()
            codes, _, _ = E(X)
            X_rec = G(codes).data.cpu().numpy()
            X = X.data.cpu().numpy()

    for k in range(5):
        fig = plot_3d_point_cloud(X[k][0], X[k][1], X[k][2],
                                  in_u_sphere=True, show=False)
        fig.savefig(
            join(results_dir, 'samples', f'{epoch}_{k}_real.png'))
        plt.close(fig)

    for k in range(5):
        fig = plot_3d_point_cloud(fake[k][0], fake[k][1], fake[k][2],
                                  in_u_sphere=True, show=False,
                                  title=str(epoch))
        fig.savefig(
            join(results_dir, 'samples', f'{epoch:05}_{k}_fixed.png'))
        plt.close(fig)

    for k in range(5):
        fig = plot_3d_point_cloud(X_rec[k][0],
                                  X_rec[k][1],
                                  X_rec[k][2],
                                  in_u_sphere=True, show=False)
        fig.savefig(join(results_dir, 'samples',
                         f'{epoch}_{k}_reconstructed.png'))
        plt.close(fig)

[0: (30)] Loss_EG: 20.8864 (REC:  12.2098 KLD:  8.6766) Time: 0:00:01.753880
[0: (60)] Loss_EG: 14.1959 (REC:  9.4017 KLD:  4.7943) Time: 0:00:02.921198
[0: (90)] Loss_EG: 7.7560 (REC:  6.7808 KLD:  0.9752) Time: 0:00:04.103354
[0: (120)] Loss_EG: 7.5078 (REC:  6.8714 KLD:  0.6364) Time: 0:00:05.297289
[0: (150)] Loss_EG: 10.6024 (REC:  10.0112 KLD:  0.5912) Time: 0:00:06.472584
[0: (180)] Loss_EG: 5.7432 (REC:  5.2815 KLD:  0.4617) Time: 0:00:07.685560
[0: (210)] Loss_EG: 6.3021 (REC:  5.8050 KLD:  0.4971) Time: 0:00:08.870005
[0: (240)] Loss_EG: 6.4879 (REC:  5.9471 KLD:  0.5408) Time: 0:00:10.031966
[0: (270)] Loss_EG: 5.9457 (REC:  5.4817 KLD:  0.4640) Time: 0:00:11.191355
[0: (300)] Loss_EG: 7.0964 (REC:  6.5950 KLD:  0.5014) Time: 0:00:12.389226
[0: (330)] Loss_EG: 5.1977 (REC:  4.6790 KLD:  0.5187) Time: 0:00:13.605834
[0: (360)] Loss_EG: 5.0736 (REC:  4.5674 KLD:  0.5062) Time: 0:00:14.804052
[0/400] Loss_G: 10.1417 Time: 0:00:14.837620
[1: (30)] Loss_EG: 4.4937 (REC:  4.0483 K

[8: (270)] Loss_EG: 2.4849 (REC:  2.1341 KLD:  0.3509) Time: 0:00:10.911659
[8: (300)] Loss_EG: 2.8343 (REC:  2.5338 KLD:  0.3005) Time: 0:00:12.045066
[8: (330)] Loss_EG: 2.7818 (REC:  2.3864 KLD:  0.3955) Time: 0:00:13.180209
[8: (360)] Loss_EG: 2.7248 (REC:  2.3894 KLD:  0.3354) Time: 0:00:14.312526
[8/400] Loss_G: 2.8242 Time: 0:00:14.346193
[9: (30)] Loss_EG: 2.5966 (REC:  2.2526 KLD:  0.3440) Time: 0:00:01.305536
[9: (60)] Loss_EG: 2.4496 (REC:  2.1241 KLD:  0.3254) Time: 0:00:02.446652
[9: (90)] Loss_EG: 2.7391 (REC:  2.4148 KLD:  0.3243) Time: 0:00:03.586953
[9: (120)] Loss_EG: 3.2718 (REC:  2.9112 KLD:  0.3606) Time: 0:00:04.719389
[9: (150)] Loss_EG: 2.9620 (REC:  2.6270 KLD:  0.3351) Time: 0:00:05.850758
[9: (180)] Loss_EG: 2.8932 (REC:  2.5212 KLD:  0.3720) Time: 0:00:06.983579
[9: (210)] Loss_EG: 2.7383 (REC:  2.3571 KLD:  0.3812) Time: 0:00:08.115631
[9: (240)] Loss_EG: 2.9491 (REC:  2.6226 KLD:  0.3264) Time: 0:00:09.248702
[9: (270)] Loss_EG: 2.6429 (REC:  2.2781 KLD:  

[17: (90)] Loss_EG: 2.2404 (REC:  1.8813 KLD:  0.3590) Time: 0:00:03.916616
[17: (120)] Loss_EG: 2.4321 (REC:  2.1857 KLD:  0.2464) Time: 0:00:05.165540
[17: (150)] Loss_EG: 2.5282 (REC:  2.2113 KLD:  0.3168) Time: 0:00:06.415503
[17: (180)] Loss_EG: 2.7841 (REC:  2.4516 KLD:  0.3325) Time: 0:00:07.665517
[17: (210)] Loss_EG: 2.2391 (REC:  1.9770 KLD:  0.2621) Time: 0:00:08.915586
[17: (240)] Loss_EG: 2.3890 (REC:  2.0943 KLD:  0.2947) Time: 0:00:10.166507
[17: (270)] Loss_EG: 2.1416 (REC:  1.7859 KLD:  0.3557) Time: 0:00:11.415731
[17: (300)] Loss_EG: 2.8016 (REC:  2.4576 KLD:  0.3440) Time: 0:00:12.665504
[17: (330)] Loss_EG: 2.7031 (REC:  2.3913 KLD:  0.3118) Time: 0:00:13.914907
[17: (360)] Loss_EG: 2.0032 (REC:  1.7483 KLD:  0.2549) Time: 0:00:15.166015
[17/400] Loss_G: 2.4004 Time: 0:00:15.199305
[18: (30)] Loss_EG: 2.1624 (REC:  1.8746 KLD:  0.2878) Time: 0:00:01.429028
[18: (60)] Loss_EG: 1.8460 (REC:  1.6122 KLD:  0.2337) Time: 0:00:02.688217
[18: (90)] Loss_EG: 2.1122 (REC:  

[25: (300)] Loss_EG: 2.4486 (REC:  2.1657 KLD:  0.2829) Time: 0:00:12.674477
[25: (330)] Loss_EG: 2.2382 (REC:  1.9496 KLD:  0.2886) Time: 0:00:13.925932
[25: (360)] Loss_EG: 2.1991 (REC:  1.9560 KLD:  0.2431) Time: 0:00:15.177629
[25/400] Loss_G: 2.0862 Time: 0:00:15.211495
[26: (30)] Loss_EG: 2.0965 (REC:  1.8321 KLD:  0.2644) Time: 0:00:01.398229
[26: (60)] Loss_EG: 1.9053 (REC:  1.6465 KLD:  0.2588) Time: 0:00:02.650834
[26: (90)] Loss_EG: 1.9773 (REC:  1.7354 KLD:  0.2419) Time: 0:00:03.896691
[26: (120)] Loss_EG: 2.0394 (REC:  1.7620 KLD:  0.2774) Time: 0:00:05.140863
[26: (150)] Loss_EG: 2.2237 (REC:  1.9620 KLD:  0.2617) Time: 0:00:06.389800
[26: (180)] Loss_EG: 1.7309 (REC:  1.4766 KLD:  0.2543) Time: 0:00:07.637862
[26: (210)] Loss_EG: 1.9628 (REC:  1.7419 KLD:  0.2210) Time: 0:00:08.884077
[26: (240)] Loss_EG: 2.0816 (REC:  1.7783 KLD:  0.3033) Time: 0:00:10.125299
[26: (270)] Loss_EG: 2.0320 (REC:  1.7666 KLD:  0.2653) Time: 0:00:11.374316
[26: (300)] Loss_EG: 1.8805 (REC: 

[34: (120)] Loss_EG: 1.9102 (REC:  1.6313 KLD:  0.2788) Time: 0:00:05.170103
[34: (150)] Loss_EG: 1.6367 (REC:  1.4344 KLD:  0.2023) Time: 0:00:06.419579
[34: (180)] Loss_EG: 1.8134 (REC:  1.5754 KLD:  0.2380) Time: 0:00:07.668756
[34: (210)] Loss_EG: 1.9254 (REC:  1.6324 KLD:  0.2930) Time: 0:00:08.918043
[34: (240)] Loss_EG: 1.6726 (REC:  1.4327 KLD:  0.2399) Time: 0:00:10.159263
[34: (270)] Loss_EG: 2.1502 (REC:  1.8936 KLD:  0.2567) Time: 0:00:11.403086
[34: (300)] Loss_EG: 1.8171 (REC:  1.5535 KLD:  0.2636) Time: 0:00:12.649899
[34: (330)] Loss_EG: 1.6170 (REC:  1.3666 KLD:  0.2505) Time: 0:00:13.891085
[34: (360)] Loss_EG: 1.7509 (REC:  1.4912 KLD:  0.2597) Time: 0:00:15.129201
[34/400] Loss_G: 1.8432 Time: 0:00:15.163819
[35: (30)] Loss_EG: 1.9687 (REC:  1.6754 KLD:  0.2932) Time: 0:00:01.530692
[35: (60)] Loss_EG: 1.9990 (REC:  1.7496 KLD:  0.2494) Time: 0:00:02.871043
[35: (90)] Loss_EG: 1.7386 (REC:  1.4907 KLD:  0.2479) Time: 0:00:04.195164
[35: (120)] Loss_EG: 1.9133 (REC: 

...

[178: (150)] Loss_EG: 1.0647 (REC:  0.8692 KLD:  0.1955) Time: 0:00:05.870485
[178: (180)] Loss_EG: 1.0557 (REC:  0.8510 KLD:  0.2047) Time: 0:00:06.994746
[178: (210)] Loss_EG: 1.0462 (REC:  0.8599 KLD:  0.1864) Time: 0:00:08.124208
[178: (240)] Loss_EG: 1.1174 (REC:  0.9236 KLD:  0.1939) Time: 0:00:09.252700
[178: (270)] Loss_EG: 1.1040 (REC:  0.9148 KLD:  0.1891) Time: 0:00:10.390603
[178: (300)] Loss_EG: 1.0474 (REC:  0.8433 KLD:  0.2041) Time: 0:00:11.561155
[178: (330)] Loss_EG: 1.2250 (REC:  1.0291 KLD:  0.1959) Time: 0:00:12.724959
[178: (360)] Loss_EG: 1.2696 (REC:  1.0639 KLD:  0.2057) Time: 0:00:13.873323
[178/400] Loss_G: 1.1440 Time: 0:00:13.906919
[179: (30)] Loss_EG: 1.1245 (REC:  0.9254 KLD:  0.1991) Time: 0:00:01.280365
[179: (60)] Loss_EG: 1.2631 (REC:  1.0681 KLD:  0.1949) Time: 0:00:02.414666
[179: (90)] Loss_EG: 1.0664 (REC:  0.8576 KLD:  0.2088) Time: 0:00:03.547071
[179: (120)] Loss_EG: 1.1681 (REC:  0.9641 KLD:  0.2040) Time: 0:00:04.678767
[179: (150)] Loss_EG:

[186: (300)] Loss_EG: 1.2388 (REC:  1.0475 KLD:  0.1913) Time: 0:00:11.471256
[186: (330)] Loss_EG: 1.2644 (REC:  1.0662 KLD:  0.1982) Time: 0:00:12.633637
[186: (360)] Loss_EG: 1.2130 (REC:  1.0145 KLD:  0.1985) Time: 0:00:13.779323
[186/400] Loss_G: 1.1355 Time: 0:00:13.818054
[187: (30)] Loss_EG: 1.0619 (REC:  0.8641 KLD:  0.1978) Time: 0:00:01.325187
[187: (60)] Loss_EG: 1.0442 (REC:  0.8506 KLD:  0.1936) Time: 0:00:02.468539
[187: (90)] Loss_EG: 1.0436 (REC:  0.8383 KLD:  0.2053) Time: 0:00:03.621871
[187: (120)] Loss_EG: 1.1436 (REC:  0.9474 KLD:  0.1962) Time: 0:00:04.769843
[187: (150)] Loss_EG: 1.2294 (REC:  1.0398 KLD:  0.1896) Time: 0:00:05.916872
[187: (180)] Loss_EG: 1.1074 (REC:  0.9102 KLD:  0.1972) Time: 0:00:07.067901
[187: (210)] Loss_EG: 1.2181 (REC:  1.0213 KLD:  0.1968) Time: 0:00:08.218623
[187: (240)] Loss_EG: 1.1540 (REC:  0.9630 KLD:  0.1911) Time: 0:00:09.366211
[187: (270)] Loss_EG: 1.1539 (REC:  0.9581 KLD:  0.1958) Time: 0:00:10.499826
[187: (300)] Loss_EG:

[195: (90)] Loss_EG: 1.1025 (REC:  0.9142 KLD:  0.1883) Time: 0:00:03.554765
[195: (120)] Loss_EG: 1.1906 (REC:  0.9897 KLD:  0.2009) Time: 0:00:04.707819
[195: (150)] Loss_EG: 1.1186 (REC:  0.9226 KLD:  0.1960) Time: 0:00:05.870270
[195: (180)] Loss_EG: 1.0115 (REC:  0.8187 KLD:  0.1927) Time: 0:00:07.090232
[195: (210)] Loss_EG: 0.9781 (REC:  0.7805 KLD:  0.1976) Time: 0:00:08.339728
[195: (240)] Loss_EG: 1.0986 (REC:  0.9000 KLD:  0.1986) Time: 0:00:09.525568
[195: (270)] Loss_EG: 1.1812 (REC:  0.9848 KLD:  0.1964) Time: 0:00:10.797995
[195: (300)] Loss_EG: 1.3662 (REC:  1.1632 KLD:  0.2031) Time: 0:00:11.975548
[195: (330)] Loss_EG: 1.0812 (REC:  0.8893 KLD:  0.1920) Time: 0:00:13.114515
[195: (360)] Loss_EG: 1.1686 (REC:  0.9752 KLD:  0.1934) Time: 0:00:14.244453
[195/400] Loss_G: 1.1298 Time: 0:00:14.278386
[196: (30)] Loss_EG: 1.0297 (REC:  0.8340 KLD:  0.1957) Time: 0:00:01.287432
[196: (60)] Loss_EG: 1.1499 (REC:  0.9571 KLD:  0.1928) Time: 0:00:02.438211
[196: (90)] Loss_EG: 

[203: (240)] Loss_EG: 1.0456 (REC:  0.8521 KLD:  0.1935) Time: 0:00:09.200743
[203: (270)] Loss_EG: 1.1253 (REC:  0.9300 KLD:  0.1953) Time: 0:00:10.326260
[203: (300)] Loss_EG: 1.0575 (REC:  0.8592 KLD:  0.1983) Time: 0:00:11.449625
[203: (330)] Loss_EG: 1.1702 (REC:  0.9622 KLD:  0.2080) Time: 0:00:12.575791
[203: (360)] Loss_EG: 1.2621 (REC:  1.0549 KLD:  0.2072) Time: 0:00:13.699525
[203/400] Loss_G: 1.1286 Time: 0:00:13.733042
[204: (30)] Loss_EG: 1.1304 (REC:  0.9341 KLD:  0.1963) Time: 0:00:01.277242
[204: (60)] Loss_EG: 1.0798 (REC:  0.8843 KLD:  0.1955) Time: 0:00:02.408779
[204: (90)] Loss_EG: 1.0711 (REC:  0.8646 KLD:  0.2065) Time: 0:00:03.545878
[204: (120)] Loss_EG: 1.1033 (REC:  0.9103 KLD:  0.1930) Time: 0:00:04.677207
[204: (150)] Loss_EG: 1.0775 (REC:  0.8780 KLD:  0.1994) Time: 0:00:05.802798
[204: (180)] Loss_EG: 1.3093 (REC:  1.0983 KLD:  0.2109) Time: 0:00:06.934680
[204: (210)] Loss_EG: 1.0738 (REC:  0.8737 KLD:  0.2001) Time: 0:00:08.066836
[204: (240)] Loss_EG:

[212: (30)] Loss_EG: 1.0422 (REC:  0.8485 KLD:  0.1937) Time: 0:00:01.310639
[212: (60)] Loss_EG: 1.0621 (REC:  0.8708 KLD:  0.1913) Time: 0:00:02.445488
[212: (90)] Loss_EG: 1.1549 (REC:  0.9436 KLD:  0.2113) Time: 0:00:03.592088
[212: (120)] Loss_EG: 1.0665 (REC:  0.8788 KLD:  0.1877) Time: 0:00:04.723272
[212: (150)] Loss_EG: 1.2938 (REC:  1.0950 KLD:  0.1988) Time: 0:00:05.852013
[212: (180)] Loss_EG: 1.1438 (REC:  0.9310 KLD:  0.2128) Time: 0:00:06.983973
[212: (210)] Loss_EG: 1.1797 (REC:  0.9784 KLD:  0.2014) Time: 0:00:08.115510
[212: (240)] Loss_EG: 1.2388 (REC:  1.0435 KLD:  0.1953) Time: 0:00:09.243081
[212: (270)] Loss_EG: 1.1091 (REC:  0.9215 KLD:  0.1876) Time: 0:00:10.372288
[212: (300)] Loss_EG: 1.1275 (REC:  0.9129 KLD:  0.2146) Time: 0:00:11.499353
[212: (330)] Loss_EG: 1.3175 (REC:  1.1212 KLD:  0.1964) Time: 0:00:12.627526
[212: (360)] Loss_EG: 1.0033 (REC:  0.8172 KLD:  0.1861) Time: 0:00:13.753228
[212/400] Loss_G: 1.1179 Time: 0:00:13.787179
[213: (30)] Loss_EG: 